In [33]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd
import os

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange


from scipy.io import loadmat

import umap

from sklearn.decomposition import PCA
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


#
from tsnecuda import TSNE

# 
np.set_printoptions(suppress=True)


Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
###############################
####### LOAD WHEEL DATA #######
###############################
root_dir = '/media/cat/256GB/donato/DON-003343/'

sessions = np.sort(os.listdir('/media/cat/256GB/donato/DON-003343/'))
session = 'DON-003343_20210303'

sessions = [session]

cmap = plt.get_cmap("viridis", 20)


for ctr,session in enumerate(sessions): 

    # 
    w = wheel.Wheel()

    #
    w.root_dir = os.path.join(root_dir, 
                              session, 
                              'TRD-2P')
    try:
        w.load_track()
    except:
        print ("session: ", session, " Missing wheel data")
        continue
        
    # print (session, np.max(w.track.distances.values), w.track.distances.values.shape[0])
        
    # 
    w.compute_velocity()

    #
    plt.plot(w.track.distances.values[::10000], 
             color=cmap(ctr),
             linewidth=3,
             label=session)
    
plt.legend()
plt.xlim(0,w.track.distances.values[::10000].shape[0])
plt.ylim(bottom=0)
plt.xlabel("Time (sec)", fontsize=20)
plt.ylabel("Distance (metres)", fontsize=20)
plt.show()


In [71]:
# # np.save('/home/cat/galvo_times.npy', w.track.galvo_triggers.times)
# # np.save('/home/cat/velocity_times.npy', w.track.velocity.times)
# # np.save('/home/cat/velocity_values.npy', w.track.velocity.values)

# print (w.track.velocity.times.shape)
# print (w.track.galvo_triggers.times.shape)

# w.plot_track(w.track.velocity)

# w.plot_track(w.track.galvo_triggers)
# plt.show()

# d = np.load('/media/cat/256GB/donato/DON-003343/DON-003343_20210223/TRD-2P/chb.npy')
# print (d.shape)

# plt.plot(d)
# plt.show()

In [3]:
#####################################
######### LOAD CALCIUM DATA #########
#####################################
#root_dir = '/media/cat/256GB/donato/DON-003343/'
#session = 'DON-003343_20210303'

#
session_dir = os.path.join(root_dir, session, 'suite2p','plane0')

c = calcium.Calcium()
c.root_dir = session_dir
c.load_suite2p()

#
c.F = c.standardize(c.F)

#
c.plot_traces(c.F, 10)


self.F (fluorescence):  (1043, 55740)
self.Fneu (neuropile):  (1043, 55740)
self.iscell (cell classifier output):  (1043, 2)
self.ops:  ()
self.spks (deconnvoved spikes):  (1043, 55740)
self.stat (footprints?):  (1043,)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 114.11it/s]


In [4]:
####################################
######### BINARIZE DATA ############
####################################
c.keep_plot = True

#
c.plot_traces(c.F, 1)

#
c.low_cutoff = .01
c.high_cutoff = 1
#F_lp = c.band_pass_filter(c.F)
c.F_lp = c.low_pass_filter(c.F)
c.plot_traces(c.F_lp,1)

#
thresh_std = 3 # number of time std 
c.F_bn, c.F_bn_aliased = c.binarize(c.F_lp, thresh_std)
c.plot_traces(c.F_bn/2.,1)




100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 356.75it/s]


In [5]:
#############################################
######### VISUALIZE [CA] and WHEEL ##########
#############################################
import matplotlib.gridspec as gridspec


fig = plt.figure()
spec = gridspec.GridSpec(ncols=1, nrows=5, figure=fig)

ax1 = fig.add_subplot(spec[0:4, 0])
ax2 = fig.add_subplot(spec[4, 0])

#
c.plot_raster(ax1, 
              c.F_bn_aliased, 
              w.track.galvo_triggers.times, 
              w.track.positions.times)

# plot track
w.plot_track(w.track.velocity, ax2, 'blue', 'velocity')

# plot histogram of mua
c.mua = wheel.empty()
c.mua.values = np.sum(c.F_bn,
                      axis=0)/100.
c.mua.times = w.track.galvo_triggers.times/10000.
w.plot_track(c.mua, ax2, 'red', 'mua')

ax2.set_xlim(w.track.distances.times[0], 
             w.track.distances.times[-1])

#
plt.show()

front padding:  3.9194 sec 117  in frame times
end padding:  25.592100000000073 sec 767  in frame times


In [35]:
# plot track
ax2=plt.subplot(111)
w.plot_track(w.track.velocity, ax2, 'blue', 'velocity')

# plot histogram of mua
c.mua = wheel.empty()
c.mua.values = np.sum(c.F_bn,axis=0)/100.
c.mua.times = w.track.galvo_triggers.times/10000.
w.plot_track(c.mua, ax2, 'red', 'mua')

#
plt.show()

In [123]:
#############################################################
#############################################################
#############################################################
#

def get_run_segments(vel,
                     times, 
                     min_vel,
                     min_run_duration,
                     min_stationary_time):
    
    # 
    vel = w.track.velocity.values.copy()
    times = w.track.velocity.times.copy()

    #
    idx = np.where(vel<min_vel)[0]
    vel[idx]=0

    # detect onsets
    diffs = vel[1:]-vel[:-1]
    idx = np.where(diffs>0)[0]
    idx2 = np.where(vel[idx+1-1]==0)
    idx3 = idx[idx2]
    init_times = times[idx3]

    diffs = vel[:-1]-vel[1:]
    idx = np.where(diffs>0)[0]
    idx2 = np.where(vel[idx+1]==0)
    idx3 = idx[idx2]
    end_times = times[idx3]

    durs = np.vstack((init_times[:-1],
                      end_times[1:])).T

    #############################################
    ####### CHECK STATIONARY TIME ###############
    #############################################
    durs2 = []
    for k in range(1, durs.shape[0], 1):
        #print (durs[k][0]-durs[k-1][1])
        if (durs[k][0]-durs[k-1][1])>=min_stationary_time:
            durs2.append(durs[k])
            #print (durs[k][0]-durs[k-1][1])
        #print ()
    #
    durs = np.array(durs2)
    
    #########################################################
    ######## CHECK MIN DURATION OF RUN ######################
    #########################################################
    durs2 = []
    for k in range(durs.shape[0]):
        #print (durs[k][1]-durs[k][0])
        if (durs[k][1]-durs[k][0])>=min_run_duration:
            durs2.append(durs[k])
            #print (durs[k][1]-durs[k][0])
            
        #print ('')

    #
    durs = np.array(durs2)

    ##############################################
    ################### PLOT #####################
    ##############################################
    fig = plt.figure()
    ax=plt.subplot(111)
    for k in range(init_times.shape[0]):
        plt.plot([init_times[k],
                  init_times[k]],
                  [0,0.3],
                  '--',
                  c='black')

    # 
    for k in range(durs.shape[0]):
        ax.axvspan(durs[k][0], 
                   durs[k][1], alpha=0.2, color='red')    

    #
    plt.plot(times, vel)
    plt.xlim(0,times[-1])
    plt.ylim(bottom=0)
    plt.show()

    return durs
    
    
#
min_vel = 0.1
min_run_duration = 0.25
min_stationary_time = 1.0
durs = get_run_segments(vel, 
                        times, 
                        min_vel,
                        min_run_duration,
                        min_stationary_time)

print ("# segements: ", durs.shape)
#print (durs)
#

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 436 and the array at index 1 has size 435